<a href="https://colab.research.google.com/github/121013jesi/magazine/blob/main/Amazon_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
         available = soup.find("div", attrs={'id':'availability'})
         available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [ ]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=laptps&crid=2AOBGX0LLRANS&sprefix=laptps%2Caps%2C528&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))


    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

<ipython-input-4-4f845fd98aaa>:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [ ]:
amazon_df

,title,price,rating,reviews,availability
0,"Acer Aspire Go 15 Slim Laptop | 15.6"" Full HD ...",,4.4 out of 5 stars,"1,244 ratings",In Stock
1,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",,4.4 out of 5 stars,"1,785 ratings",In Stock
2,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,,4.6 out of 5 stars,"39,164 ratings",In Stock
3,"HP Newest 14"" Ultral Light Laptop for Students...",,4.5 out of 5 stars,"2,049 ratings",In Stock
4,"HP Newest 255 G10 Laptop for Home or Work, 16G...",,4.5 out of 5 stars,80 ratings,In Stock
5,"NIMO 15.6 FHD Student Laptop, 16GB RAM, 1TB SS...",,4.5 out of 5 stars,184 ratings,In Stock
6,"Lenovo IdeaPad 1 Student Laptop, Intel Dual Co...",,4.4 out of 5 stars,750 ratings,Only 6 left in stock - order soon.
7,"HP Portable Laptop, Student and Business, 14"" ...",,4.5 out of 5 stars,"1,571 ratings",In Stock
8,"HP 15.6"" Portable Laptop (Include 1 Year Micro...",,4.6 out of 5 stars,639 ratings,Only 4 left in stock - order soon.
9,Dell Inspiron 15 3000 3520 Business Laptop Com...,,4.4 out of 5 stars,158 ratings,Only 6 left in stock - order soon.
